In [107]:
from pyspark.sql import SparkSession
import os
import sys
#from config import API_KEY
from datetime import datetime, timedelta
import requests
from pyspark.sql.types import StructType, StructField, StringType, FloatType, TimestampType, DateType, DoubleType, IntegerType
import pandas as pd
from pyspark.sql import Row
from pyspark.sql.window import Window
from pyspark.sql.functions import dense_rank, col, rank, avg, round, count, countDistinct, udf, when, lower, row_number, regexp_extract
from pyspark.sql import functions as F
import sqlite3
from pyspark.sql.functions import date_format

In [3]:
print(f"JAVA_HOME: {os.environ.get('JAVA_HOME')}")
print(f"PATH: {os.environ.get('PATH')}")
os.environ['JAVA_HOME'] = r'C:\Program Files\Java\jdk-1.8'
print(f"JAVA_HOME: {os.environ.get('JAVA_HOME')}")

#os.environ["PYSPARK_PYTHON"] = r"C:\Users\lucas\anaconda3\python.exe"
#os.environ["PYSPARK_DRIVER_PYTHON"] = r"C:\Users\lucas\anaconda3\python.exe"

JAVA_HOME: C:\Program Files\Java\jdk-1.8
PATH: c:\Users\lucas\anaconda3\envs\pyspark_env;C:\Users\lucas\anaconda3\envs\pyspark_env;C:\Users\lucas\anaconda3\envs\pyspark_env\Library\mingw-w64\bin;C:\Users\lucas\anaconda3\envs\pyspark_env\Library\usr\bin;C:\Users\lucas\anaconda3\envs\pyspark_env\Library\bin;C:\Users\lucas\anaconda3\envs\pyspark_env\Scripts;C:\Users\lucas\anaconda3\envs\pyspark_env\bin;C:\Users\lucas\anaconda3\condabin;C:\Users\lucas\anaconda3;C:\Program Files (x86)\Common Files\Oracle\Java\java8path;C:\Program Files (x86)\Common Files\Oracle\Java\javapath;C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.8\libnvvp;C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.8\bin;C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.1\bin;C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.1\libnvvp;C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.6\bin;C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.6\libnvvp;C:\Program Files\Rockwell Software\RSCommon;C

In [4]:
spark = SparkSession.builder \
    .appName("movieapp") \
    .getOrCreate()

In [5]:
tag_df = spark.read.csv("tag.csv", header=True)

In [8]:
tag_df.show()


+------+-------+-----------------+-------------------+
|userId|movieId|              tag|          timestamp|
+------+-------+-----------------+-------------------+
|    18|   4141|      Mark Waters|2009-04-24 18:19:40|
|    65|    208|        dark hero|2013-05-10 01:41:18|
|    65|    353|        dark hero|2013-05-10 01:41:19|
|    65|    521|    noir thriller|2013-05-10 01:39:43|
|    65|    592|        dark hero|2013-05-10 01:41:18|
|    65|    668|        bollywood|2013-05-10 01:37:56|
|    65|    898| screwball comedy|2013-05-10 01:42:40|
|    65|   1248|    noir thriller|2013-05-10 01:39:43|
|    65|   1391|             mars|2013-05-10 01:40:55|
|    65|   1617|         neo-noir|2013-05-10 01:43:37|
|    65|   1694|            jesus|2013-05-10 01:38:45|
|    65|   1783|    noir thriller|2013-05-10 01:39:43|
|    65|   2022|            jesus|2013-05-10 01:38:45|
|    65|   2193|           dragon|2013-05-10 02:01:54|
|    65|   2353|conspiracy theory|2013-05-10 02:01:06|
|    65|  

In [ ]:
#check for duplicates among userid movieid pairing 
tag_df_dup = tag_df.groupby(['userId','movieId']).count().filter(col('count') > 1)
tag_df_dup.show()
check = tag_df.filter(col('userId') == 342)
check.show()
#duplicates look legitimate as different tags from the same user on the same movie


+------+-------+-----+
|userId|movieId|count|
+------+-------+-----+
|   342|   6953|    3|
|   469|    290|    2|
|   631|   3578|    4|
|  1629|     50|    5|
|  1629|    593|    2|
|  1662|   4226|    7|
|  1741|   5049|    2|
|  1741|  49234|    2|
|  1741|  53129|    5|
|  1741|  96052|    3|
|  2193|  55269|    6|
|  2274|  55820|    2|
|  2299|     34|    5|
|  2299|    316|    6|
|  2423|  56788|    2|
|  2611|     47|    3|
|  2751|   4847|    3|
|  2931|   2502|    2|
|  3576|    910|   10|
|  3596|   8833|    2|
+------+-------+-----+
only showing top 20 rows

+------+-------+----------------+-------------------+
|userId|movieId|             tag|          timestamp|
+------+-------+----------------+-------------------+
|   342|     32|post-apocalyptic|2012-01-25 18:02:10|
|   342|     32|      psychology|2012-01-25 18:02:15|
|   342|     32|     time travel|2012-01-25 18:02:06|
|   342|     50| organized crime|2012-01-31 16:58:31|
|   342|     50|    twist ending|2012-01-31 

In [49]:
#check for null values
null_counts = {}
for column in tag_df.columns:
    null_count = tag_df.filter(col(column).isNull()).count()
    null_counts[column] = null_count
print(null_counts)

{'userId': 0, 'movieId': 0, 'tag': 0, 'timestamp': 3}


In [69]:
#make date column
tag_df = tag_df.withColumn('tag_date',date_format('timestamp','yyyy-MM-dd'))
tag_df.show()

+------+-------+-----------------+-------------------+----------+
|userId|movieId|              tag|          timestamp|  tag_date|
+------+-------+-----------------+-------------------+----------+
|    18|   4141|      Mark Waters|2009-04-24 18:19:40|2009-04-24|
|    65|    208|        dark hero|2013-05-10 01:41:18|2013-05-10|
|    65|    353|        dark hero|2013-05-10 01:41:19|2013-05-10|
|    65|    521|    noir thriller|2013-05-10 01:39:43|2013-05-10|
|    65|    592|        dark hero|2013-05-10 01:41:18|2013-05-10|
|    65|    668|        bollywood|2013-05-10 01:37:56|2013-05-10|
|    65|    898| screwball comedy|2013-05-10 01:42:40|2013-05-10|
|    65|   1248|    noir thriller|2013-05-10 01:39:43|2013-05-10|
|    65|   1391|             mars|2013-05-10 01:40:55|2013-05-10|
|    65|   1617|         neo-noir|2013-05-10 01:43:37|2013-05-10|
|    65|   1694|            jesus|2013-05-10 01:38:45|2013-05-10|
|    65|   1783|    noir thriller|2013-05-10 01:39:43|2013-05-10|
|    65|  

In [9]:
rating_df = spark.read.csv("rating.csv",header=True)

In [10]:
rating_df.show()

+------+-------+------+-------------------+
|userId|movieId|rating|          timestamp|
+------+-------+------+-------------------+
|     1|      2|   3.5|2005-04-02 23:53:47|
|     1|     29|   3.5|2005-04-02 23:31:16|
|     1|     32|   3.5|2005-04-02 23:33:39|
|     1|     47|   3.5|2005-04-02 23:32:07|
|     1|     50|   3.5|2005-04-02 23:29:40|
|     1|    112|   3.5|2004-09-10 03:09:00|
|     1|    151|     4|2004-09-10 03:08:54|
|     1|    223|     4|2005-04-02 23:46:13|
|     1|    253|     4|2005-04-02 23:35:40|
|     1|    260|     4|2005-04-02 23:33:46|
|     1|    293|     4|2005-04-02 23:31:43|
|     1|    296|     4|2005-04-02 23:32:47|
|     1|    318|     4|2005-04-02 23:33:18|
|     1|    337|   3.5|2004-09-10 03:08:29|
|     1|    367|   3.5|2005-04-02 23:53:00|
|     1|    541|     4|2005-04-02 23:30:03|
|     1|    589|   3.5|2005-04-02 23:45:57|
|     1|    593|   3.5|2005-04-02 23:31:01|
|     1|    653|     3|2004-09-10 03:08:11|
|     1|    919|   3.5|2004-09-1

In [ ]:
#check for duplicates
rating_df_dup = rating_df.groupby(['userId','movieId']).count().orderBy('count',ascending=False)
rating_df_dup.show()
#no duplicates

In [61]:
#check for nulls
null_counts = {}
for column in rating_df.columns:
    null_count = rating_df.filter(col(column).isNull()).count()
    null_counts[column] = null_count

In [67]:
rating_df = rating_df.withColumn('rating_date',date_format('timestamp','yyy-MM-dd'))
rating_df.show()

+------+-------+------+-------------------+-----------+
|userId|movieId|rating|          timestamp|rating_date|
+------+-------+------+-------------------+-----------+
|     1|      2|   3.5|2005-04-02 23:53:47| 2005-04-02|
|     1|     29|   3.5|2005-04-02 23:31:16| 2005-04-02|
|     1|     32|   3.5|2005-04-02 23:33:39| 2005-04-02|
|     1|     47|   3.5|2005-04-02 23:32:07| 2005-04-02|
|     1|     50|   3.5|2005-04-02 23:29:40| 2005-04-02|
|     1|    112|   3.5|2004-09-10 03:09:00| 2004-09-10|
|     1|    151|     4|2004-09-10 03:08:54| 2004-09-10|
|     1|    223|     4|2005-04-02 23:46:13| 2005-04-02|
|     1|    253|     4|2005-04-02 23:35:40| 2005-04-02|
|     1|    260|     4|2005-04-02 23:33:46| 2005-04-02|
|     1|    293|     4|2005-04-02 23:31:43| 2005-04-02|
|     1|    296|     4|2005-04-02 23:32:47| 2005-04-02|
|     1|    318|     4|2005-04-02 23:33:18| 2005-04-02|
|     1|    337|   3.5|2004-09-10 03:08:29| 2004-09-10|
|     1|    367|   3.5|2005-04-02 23:53:00| 2005

In [11]:
movie_df = spark.read.csv("movie.csv",header=True)

In [12]:
movie_df.show()

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Adventure|Animati...|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
|     16|       Casino (1995)|         Crime|Drama|
|     17|Sen

In [73]:
#check for duplicate movieId
movie_df_dup = movie_df.groupby('movieId').count().filter(col('count') > 1)
movie_df_dup.show()

+-------+-----+
|movieId|count|
+-------+-----+
+-------+-----+



In [74]:
#check for null
null_counts = {}
for column in movie_df.columns:
    null_count = movie_df.filter(col(column).isNull()).count()
    null_counts[column] = null_count
print(null_counts)

{'movieId': 0, 'title': 0, 'genres': 0}


In [14]:
link_df = spark.read.csv("link.csv",header=True)

In [15]:
link_df.show()

+-------+------+------+
|movieId|imdbId|tmdbId|
+-------+------+------+
|      1|114709|   862|
|      2|113497|  8844|
|      3|113228| 15602|
|      4|114885| 31357|
|      5|113041| 11862|
|      6|113277|   949|
|      7|114319| 11860|
|      8|112302| 45325|
|      9|114576|  9091|
|     10|113189|   710|
|     11|112346|  9087|
|     12|112896| 12110|
|     13|112453| 21032|
|     14|113987| 10858|
|     15|112760|  1408|
|     16|112641|   524|
|     17|114388|  4584|
|     18|113101|     5|
|     19|112281|  9273|
|     20|113845| 11517|
+-------+------+------+
only showing top 20 rows



In [75]:
link_df_dup = link_df.groupby('movieId').count().filter(col('count') > 1)
link_df_dup.show()

+-------+-----+
|movieId|count|
+-------+-----+
+-------+-----+



In [76]:
#check for nulls 
null_counts = {}
for column in link_df.columns:
    null_count = link_df.filter(col(column).isNull()).count()
    null_counts[column] = null_count
print(null_counts)

{'movieId': 0, 'imdbId': 0, 'tmdbId': 252}


In [16]:
genome_tags_df = spark.read.csv("genome_tags.csv",header=True)

In [17]:
genome_tags_df.show()

+-----+---------------+
|tagId|            tag|
+-----+---------------+
|    1|            007|
|    2|   007 (series)|
|    3|   18th century|
|    4|          1920s|
|    5|          1930s|
|    6|          1950s|
|    7|          1960s|
|    8|          1970s|
|    9|          1980s|
|   10|   19th century|
|   11|             3d|
|   12|           70mm|
|   13|            80s|
|   14|           9/11|
|   15|        aardman|
|   16|aardman studios|
|   17|       abortion|
|   18|         absurd|
|   19|         action|
|   20|  action packed|
+-----+---------------+
only showing top 20 rows



In [ ]:
#check for dups
genome_tags_df_dup = genome_tags_df.groupby('tagId').count().filter(col('count')>1)
genome_tags_df_dup.show()

+-----+-----+
|tagId|count|
+-----+-----+
+-----+-----+



In [78]:
null_counts = {}
for column in genome_tags_df.columns:
    null_count = genome_tags_df.filter(col(column).isNull()).count()
    null_counts[column] = null_count
print(null_counts)

{'tagId': 0, 'tag': 0}


In [18]:
genome_scores_df = spark.read.csv("genome_scores.csv",header=True)

In [19]:
genome_scores_df.show()

+-------+-----+-------------------+
|movieId|tagId|          relevance|
+-------+-----+-------------------+
|      1|    1|              0.025|
|      1|    2|              0.025|
|      1|    3|            0.05775|
|      1|    4|            0.09675|
|      1|    5|            0.14675|
|      1|    6|              0.217|
|      1|    7|              0.067|
|      1|    8|            0.26275|
|      1|    9|              0.262|
|      1|   10|              0.032|
|      1|   11|              0.577|
|      1|   12|            0.11625|
|      1|   13|              0.188|
|      1|   14|0.00800000000000001|
|      1|   15|            0.03675|
|      1|   16|            0.28175|
|      1|   17|0.00700000000000001|
|      1|   18|             0.1105|
|      1|   19|             0.6705|
|      1|   20|             0.1845|
+-------+-----+-------------------+
only showing top 20 rows



In [87]:
top_tag_df.count()

465564

In [91]:
top_tag_df.count()

465564

In [90]:
#since there can be multiple tags for each movie by a user we will only take the most relevant tag so that we can remove duplication on final table joins
top_tag_df = tag_df.join(genome_tags_df,tag_df.tag == genome_tags_df.tag,how='left').select(tag_df.userId,tag_df.movieId,tag_df.tag,genome_tags_df.tagId)
top_tag_df = top_tag_df.join(genome_scores_df,(top_tag_df.tagId == genome_scores_df.tagId) & (top_tag_df.movieId == genome_scores_df.movieId),how='left').select(top_tag_df['*'],genome_scores_df.relevance)
top_tag_df.show()

+------+-------+--------------------+-----+---------+
|userId|movieId|                 tag|tagId|relevance|
+------+-------+--------------------+-----+---------+
| 78917|   1235|         Cat Stevens| NULL|     NULL|
|131900|  80860|real life situations| NULL|     NULL|
| 78917|   1285|        black comedy|  141|  0.86375|
|    65|   5135|           bollywood|  149|  0.94875|
|    65|   1694|               jesus|  579|  0.94125|
|    65|   2840|               jesus|  579|  0.97175|
|    65|   3052|               jesus|  579|  0.96325|
|    65|   2726|       noir thriller|  712|   0.9945|
| 22963|   4878|             surreal|  995|  0.93325|
| 78917|   1227|     Ennio Morricone| NULL|     NULL|
|110507|   3148|           Seen 2009| NULL|     NULL|
|110507|   3207|           Seen 2013| NULL|     NULL|
|110507|   3379|           seen 2012| NULL|     NULL|
|110507|   3379| seen more than once| NULL|     NULL|
| 22963|   4019|       inspirational|  548|  0.89125|
|    65|   1248|       noir 

In [97]:
#window function continuation
window_spec = Window.partitionBy(['userId','movieId']).orderBy(col('relevance').desc())
top_tag_ranked_df = top_tag_df.withColumn('row_number',row_number().over(window_spec)).filter(col('row_number') == 1)
top_tag_ranked_df.show()

+------+-------+--------------------+-----+---------+----------+
|userId|movieId|                 tag|tagId|relevance|row_number|
+------+-------+--------------------+-----+---------+----------+
|100144|    172|           cyberpunk|  280|  0.99975|         1|
|100144|  44191|    based on a comic|  108|    0.973|         1|
| 10017|   2371|               silly|  919|    0.634|         1|
|100366| 102066|         Dog killing| NULL|     NULL|         1|
|100366| 103483|       child killing| NULL|     NULL|         1|
|100366|    150|               tense| 1020|   0.9655|         1|
|100366|   2700|  creative profanity| NULL|     NULL|         1|
|100366|   2787|           anthology| NULL|     NULL|         1|
|100366|    393|based on a video ...|  112|  0.99975|         1|
|100366|  44974|             torture| 1034|  0.88625|         1|
|100366|   5585|              trolls| NULL|     NULL|         1|
|100366|  60766|           uplifting| NULL|     NULL|         1|
|100366|   6214|         

In [98]:
df1 = rating_df.join(top_tag_ranked_df, (rating_df['userId'] == top_tag_ranked_df['userId']) & (rating_df['movieId'] == top_tag_ranked_df['movieId']),how='left').select(rating_df['userId'],rating_df['movieId'],rating_df.rating,top_tag_ranked_df['tag'])

In [101]:
df1.show()

+------+-------+------+----+
|userId|movieId|rating| tag|
+------+-------+------+----+
|     1|    919|   3.5|NULL|
|105081|   4306|     3|NULL|
|116465|   3100|     4|NULL|
| 11731|   3257|     2|NULL|
| 11731|  58559|     1|NULL|
| 11732|      1|     4|NULL|
|127936|   2448|     3|NULL|
|127936|   4718|     3|NULL|
| 23312|   1484|     4|NULL|
| 23312|   2111|     1|NULL|
| 35001|   3354|     2|NULL|
| 35001|   3676|     3|NULL|
| 46736|   1204|     4|NULL|
| 58552|  48780|     5|NULL|
| 58552|  55765|   2.5|NULL|
| 70324|   2738|     3|NULL|
| 81744|   1285|     2|NULL|
| 81744|   1295|     4|NULL|
| 81744|    357|     3|NULL|
| 81744|    365|     5|NULL|
+------+-------+------+----+
only showing top 20 rows



In [ ]:
#filtered_df1 = df1.filter(df1['tag'].isNotNull())

In [28]:
#filtered_df1.show()

In [104]:
df2 = df1.join(movie_df,df1['movieId'] == movie_df['movieId'], how='left').select(df1['userId'],df1.movieId,df1.rating,df1.tag,movie_df.title,movie_df.genres)

In [106]:
df2.show()

+------+-------+------+----+--------------------+--------------------+
|userId|movieId|rating| tag|               title|              genres|
+------+-------+------+----+--------------------+--------------------+
|     1|    919|   3.5|NULL|Wizard of Oz, The...|Adventure|Childre...|
|105081|   4306|     3|NULL|        Shrek (2001)|Adventure|Animati...|
|116465|   3100|     4|NULL|River Runs Throug...|               Drama|
| 11731|   3257|     2|NULL|Bodyguard, The (1...|Drama|Romance|Thr...|
| 11731|  58559|     1|NULL|Dark Knight, The ...|Action|Crime|Dram...|
| 11732|      1|     4|NULL|    Toy Story (1995)|Adventure|Animati...|
|127936|   2448|     3|NULL|        Virus (1999)|       Horror|Sci-Fi|
|127936|   4718|     3|NULL|American Pie 2 (2...|              Comedy|
| 23312|   1484|     4|NULL|Daytrippers, The ...|Comedy|Drama|Myst...|
| 23312|   2111|     1|NULL|Man with Two Brai...|              Comedy|
| 35001|   3354|     2|NULL|Mission to Mars (...|              Sci-Fi|
| 3500

In [108]:
#extract year from the title
pattern = r"\((\d{4})\)"
df2 = df2.withColumn('year',regexp_extract('title',pattern,1))

In [109]:
df2.show()

+------+-------+------+----+--------------------+--------------------+----+
|userId|movieId|rating| tag|               title|              genres|year|
+------+-------+------+----+--------------------+--------------------+----+
|     1|    919|   3.5|NULL|Wizard of Oz, The...|Adventure|Childre...|1939|
|105081|   4306|     3|NULL|        Shrek (2001)|Adventure|Animati...|2001|
|116465|   3100|     4|NULL|River Runs Throug...|               Drama|1992|
| 11731|   3257|     2|NULL|Bodyguard, The (1...|Drama|Romance|Thr...|1992|
| 11731|  58559|     1|NULL|Dark Knight, The ...|Action|Crime|Dram...|2008|
| 11732|      1|     4|NULL|    Toy Story (1995)|Adventure|Animati...|1995|
|127936|   2448|     3|NULL|        Virus (1999)|       Horror|Sci-Fi|1999|
|127936|   4718|     3|NULL|American Pie 2 (2...|              Comedy|2001|
| 23312|   1484|     4|NULL|Daytrippers, The ...|Comedy|Drama|Myst...|1996|
| 23312|   2111|     1|NULL|Man with Two Brai...|              Comedy|1983|
| 35001|   3

In [116]:
df2 = df2.withColumn("genres",F.split(F.col("genres"),"\|"))

In [ ]:
unique_genres = df2.select(F.explode(F.col("genres")).alias("genre")).distinct().collect()


In [120]:
unique_genres_list = [row['genre'] for row in unique_genres]

In [122]:
for genre in unique_genres_list:
    df2 = df2.withColumn(genre,F.when(F.array_contains(F.col("genres"),genre),1).otherwise(0))

In [124]:
df2 = df2.drop("genres")

In [126]:
#remove year from title
pattern = r"\s\(\d{4}\)"
df2 = df2.withColumn("title",F.regexp_replace(F.col("title"),pattern,""))

In [127]:
df2.show()

+------+-------+------+----+--------------------+----+-----+-------+--------+---------+-----+---+-----------+-------+-------+-------+---------+---------+------------------+----+------+-------+------+--------+------+------+
|userId|movieId|rating| tag|               title|year|Crime|Romance|Thriller|Adventure|Drama|War|Documentary|Fantasy|Mystery|Musical|Animation|Film-Noir|(no genres listed)|IMAX|Horror|Western|Comedy|Children|Action|Sci-Fi|
+------+-------+------+----+--------------------+----+-----+-------+--------+---------+-----+---+-----------+-------+-------+-------+---------+---------+------------------+----+------+-------+------+--------+------+------+
|     1|   1201|     3|NULL|Good, the Bad and...|1966|    0|      0|       0|        1|    0|  0|          0|      0|      0|      0|        0|        0|                 0|   0|     0|      1|     0|       0|     1|     0|
|     1|   1219|     4|NULL|              Psycho|1960|    1|      0|       0|        0|    0|  0|          0

In [130]:
average_movie_rating = df2.groupby('title').agg(avg('rating').alias('avg_rating')).orderBy('avg_rating',ascending=False)
average_movie_rating.show()
    

+--------------------+----------+
|               title|avg_rating|
+--------------------+----------+
|Who Killed Vincen...|       5.0|
|          Sierra, La|       5.0|
| The Floating Castle|       5.0|
|       Boy Meets Boy|       5.0|
|            Freeheld|       5.0|
|Paying the Price:...|       5.0|
|   Lady of Chance, A|       5.0|
|            Mutantes|       5.0|
|         When I Walk|       5.0|
|Bill Hicks: Sane Man|       5.0|
|            Giorgino|       5.0|
|       Boys (Drenge)|       5.0|
|        Al otro lado|       5.0|
|Blue Swallow (Che...|       5.0|
|       The Encounter|       5.0|
|New Rulers of the...|       5.0|
|   The Wrecking Crew|       5.0|
|Prom Queen: The M...|       5.0|
|           Abendland|       5.0|
|Peace, Propaganda...|       5.0|
+--------------------+----------+
only showing top 20 rows

